# Is Monday Effect an Urban Myth?

**Authors**: Blythe King, Dinggyue Lie, Lucy (Yu) Xue\*, Sungbin Youk\
**Date**: May 30th, 2021\
**Description**: As a final project for PSTAT 234 in University of California, Santa Barbara, the authors examined the presence of Monday effect. \
The authors equally contributed to the project. The names of the authors are in an alphabetical order. The corresponding author is indicated with *.
----

**Table of Contents**
1. [Introduction](#introduction)\
    A. [Predicting the Stock Market](#predicting-the-stock-market)\
    B. [What is Monday Effect](#what-is-monday-effect)\
    C. [Our Objectives](#our-objectives)
2. [Tackling Objective 1](#tackling-objective-1)
3. [Tackling Objective 2](#tackling-objective-2)

## Introduction<a clas ="anchor" id = "introduction"></a>

### Predicting the Stock Market <a clas ="anchor" id = "predicting-the-stock-market"></a>

It will be great if you can predict the changes in the stock market. It will make you rich. Isn't that everyone's dream? Unfortunately, [efficient market hypothesis](https://www.investopedia.com/terms/e/efficientmarkethypothesis.asp) postulates that generating a stable parameter that reflects the share prices is impossible as the share prices reflect all information. 

It would be against the efficient market hypothesis if there is a predictable *pattern* in the stock market. In 1973, [Frank Cross](https://www.jstor.org/stable/pdf/4529641.pdf?refreqid=excelsior%3Adeff8e6e9e2c4c0b275b4b03a21b9c13) documented a non-random movement in stock prices. Here are the main findings from examining the Standard & Poor's Composite Stock Index from 1953 to 1970:
- The index have risen on Friday more often than on any other days of the week, and have risen least often on Monday. 
- When the Friday index declined, the Monday index was more likely to also see a decline. When the Friday index advanced, the Monday index was likely to remain static (neither advancing nor declining). 

### What is Monday Effect? <a clas ="anchor" id = "what-is-monday-effect"></a>

Over the years, Frank Cross's findings were coined into what is now known as the **Monday Effect**. There are two different definitions of the monday effect (each corresponding to the two findings that are mentioned above). 

- Monday effect states that the returns on Monday are less than the other days of the week, and are often negative on average ([Pettengill, 2003](https://www.jstor.org/stable/pdf/23292837.pdf?refreqid=excelsior%3A6da162ff7d91746d901fc154171e6015)).
- Monday effect states that the returns on the stock market on Monday, especially the first few hours, will follow the pattern of the previous Friday, espeically the last few hours ([Investopedia](https://www.investopedia.com/terms/m/mondayeffect.asp)). 

You may wonder what may be the reason behind this abnormality in the stock prices. As the existence of Monday effect is controversial (thus, the reason for our project), there isn't a clear answer. Some state that the stock returns are low on Monday because companies may hold on to bad news until the last day of stock trading (Friday), which in turn makes the next stock trading day (Monday) to take the hit. 

### Our Objectives <a clas ="anchor" id = "our-objectives"></a>

The objective of our project is in two-folds:
1) [Arman and Lestari](https://www.atlantis-press.com/proceedings/icame-18/125917114) examined the Monday effect (the first definition) in the Indonesian Stock Exchange. In their study, the first definition of Monday effect is used: The stock returns of Monday is less than the other days of the week. We will first examine if Monday effect is also present in the U.S. stock market.\
2) The second objective of this study is to examine the second definition of Monday effect: Monday's returns are correlated to that of Friday.\
3) We will take a step further and apply time series analysis.

----
## Tackling Objective 1 <a clas ="anchor" id = "tackling-objective-1"></a>

In our analysis, the stock returns of S&P 500 from 2014 to 2017 are examined. The stock return data are obtained from yfinance package in python.

### Summary of Arman and Lestari's Study

Arman and Lestari examined the Monday effect by examining the banking sectors on the Indonesian stock market from 2014 to 2017. A one-sample t-test was conducted for each of the weekdays. The results indicated that the average stock return on Monday is -0.0006, which was not statistically significant. 

### Importing Libraries and Packages

In [74]:
import yfinance as yf
import numpy as np
import pandas as pd
import requests
import datetime
from datetime import date
import calendar
import io
from scipy import stats
!pip install openpyxl

### Importing the list of ticker for S&P 500 between 2014 to 2017

The first step is to retrieve the companies that constituted S&P 500 in the past. 

In [2]:
# Downloading the csv file from a Github page which has a list of companies and when they were added or removed from S&P 500
url = "https://raw.githubusercontent.com/leosmigel/analyzingalpha/master/sp500-historical-components-and-changes/sp500_history.csv"
download = requests.get(url).content

# Reading the downloaded content and turning it into a pandas dataframe
df = pd.read_csv(io.StringIO(download.decode('utf-8')))

#Turning the date column into a datetime object
df["date"] = pd.to_datetime(df["date"])

# Printing out the first 5 rows of the dataframe
df.head()

,Unnamed: 0,cik,date,name,value,variable
0,183,72741.0,1957-01-01,Eversource Energy,ES,added_ticker
1,228,874766.0,1957-01-01,Hartford Financial Svc.Gp.,HIG,added_ticker
2,435,1113169.0,1957-01-01,T. Rowe Price Group,TROW,added_ticker
3,349,1111711.0,1957-01-01,NiSource Inc.,NI,added_ticker
4,185,1109357.0,1957-01-01,Exelon Corp.,EXC,added_ticker


In [3]:
# Function to retrieve the tickers in S&P 500 for a given timeframe
def past_SP_ticker(end_date):
    ticker_list = []
    global df
    for index,row in df.iterrows():
        if row['date'] > end_date:
            break
        else:
            if row['variable'] == "added_ticker":
                ticker_list.append(row['value'])
            elif row['value'] in ticker_list:
                ticker_list.remove(row['value'])
    return ticker_list

In [4]:
# Using the past_SP_ticker() function to retrieve the tickers of S&P 500 for 2017. 
end_date = '20171231'
date_time_obj = datetime.datetime.strptime(end_date,'%Y%m%d')
SP_ticker_2017 = past_SP_ticker(date_time_obj)

### Creating a dataframe of stock returns for the identified S&P 500 constituents of 2017
The next step is to obtain the daily stock returns of the selected companies. This requires several steps: obtain the stock data of the S&P 500 constituents of 2017, delete the missing values, calculate the log retruns, create a multilevel index (i.e., hierarchical index) with the days of the week

#### Obtaining the stock data of S&P 500 constituents of 2017

In [5]:
# Using the ticker to obtain stock prices from yfinance
rawdata = yf.download(SP_ticker_2017, start="2013-12-31", end="2017-12-31")
rawdata.columns = rawdata.columns.set_names(['Value', 'Symbol'])
rawdata.head()

[*********************100%***********************]  488 of 488 completed

33 Failed downloads:
- BRK.B: No data found, symbol may be delisted
- TIF: No data found, symbol may be delisted
- JEC: No data found, symbol may be delisted
- FOX: Data doesn't exist for startDate = 1388448000, endDate = 1514678400
- RTN: No data found, symbol may be delisted
- SYMC: No data found, symbol may be delisted
- BHGE: No data found, symbol may be delisted
- KFT: No data found for this date range, symbol may be delisted
- CTL: No data found, symbol may be delisted
- BF.B: No data found for this date range, symbol may be delisted
- ETFC: No data found, symbol may be delisted
- STI: No data found, symbol may be delisted
- GGP: No data found for this date range, symbol may be delisted
- PCLN: No data found for this date range, symbol may be delisted
- CBS: No data found, symbol may be delisted
- TYC: No data found for this date range, symbol may be delisted
- VIAB: No data found, symbol may be delisted
- 

Value       Adj Close                                                          \
Symbol              A        AAL         AAP       AAPL       ABBV        ABC   
Date                                                                            
2013-12-30        NaN        NaN         NaN        NaN        NaN        NaN   
2013-12-31  38.247646  23.804220  108.574318  17.819059  38.783775  62.277599   
2014-01-02  37.592243  23.907927  107.652184  17.568451  38.174229  61.905617   
2014-01-03  38.067078  25.020357  110.732445  17.182550  38.409229  61.949898   
2014-01-06  37.879810  25.482304  109.672997  17.276245  37.006523  61.728436   

Value                                                   ...     Volume  \
Symbol            ABT        ACN       ADBE        ADI  ...        XEL   
Date                                                    ...              
2013-12-30        NaN        NaN        NaN        NaN  ...        NaN   
2013-12-31  33.028294  71.508987  59.880001  42.959011  ...  1752800.0   
2014-01-02  32.942123  70.560982  59.290001  41.567257  ...  3192300.0   
2014-01-03  33.295418  70.795822  59.160000  41.845615  ...  2939400.0   
2014-01-06  33.734875  70.047867  58.119999  41.609428  ...  3382300.0   

Value                                                                         \
Symbol           XLNX         XOM       XRAY        XRX       XYL        YUM   
Date                                                                           
2013-12-30        NaN         NaN        NaN        NaN       NaN        NaN   
2013-12-31  1215400.0   8509600.0   434400.0  2033400.0  558000.0  2966800.0   
2014-01-02  3436800.0  11028100.0  1025400.0  3977600.0  765100.0  2721200.0   
2014-01-03  1982700.0   9295600.0   623300.0  2763700.0  454500.0  2026800.0   
2014-01-06  1970800.0  11848500.0   986700.0  5657100.0  849400.0  4083600.0   

Value                                        
Symbol            ZBH       ZION        ZTS  
Date                                         
2013-12-30        NaN        NaN        NaN  
2013-12-31   650000.0  1077400.0  2270400.0  
2014-01-02   868800.0  1356700.0  2576100.0  
2014-01-03  1288200.0  1122500.0  2524900.0  
2014-01-06  1414900.0  1988200.0  2763200.0  

[5 rows x 2928 columns]

#### Deleting the missing values

In [6]:
# Inspecting the missing values in terms of rows
rawdata['Close'].isna().sum(axis=0).describe()

count     488.000000
mean       82.971311
std       262.765093
min         3.000000
25%         3.000000
50%         3.000000
75%         3.000000
max      1011.000000
dtype: float64

In [7]:
# Making an list of tuples for tickers that has more than 3 missing values
high_missing_ticker = rawdata['Close'].isna().sum(axis=0) > 3
high_missing_ticker_list = high_missing_ticker[high_missing_ticker].index.tolist()
high_missing_ticker_tuples = list()
for i in ['Adj Close', 'Open', 'Close', 'High' ,'Low', 'Volume']:
    high_missing_ticker_tuples += list(zip([i]*len(high_missing_ticker_list),high_missing_ticker_list))

In [8]:
# Excluding columns (i.e., tickers) that has more than 3 missing values 
rawdata = rawdata.drop(high_missing_ticker_tuples, axis = 1)

In [9]:
# Finding out the dates that all tickers (columns) have missing values
missingdate =rawdata.isna().sum(axis=1) > 0
missingdate[missingdate].index

DatetimeIndex(['2013-12-30', '2016-01-18', '2017-02-20'], dtype='datetime64[ns]', name='Date', freq=None)

In [10]:
# row with the index of 2013-12-31 will be deleted as it is out of the scope of our data (2014~2017)
rawdata = rawdata.drop(pd.Timestamp('2013-12-30'))

In [11]:
# rows with the index of 2017-01-02 and 2017-02-20 are replaced with the values from the previous date
rawdata = rawdata.fillna(method= 'ffill')

In [12]:
# Double check to see if all the missing values were either removed or replaced
(rawdata.isna().sum(axis=None)>0).any()

False

#### Calculating the log returns for closing price

In [13]:
# Getting the log returns from stock prices
logret = np.log(rawdata['Close']).diff()
logret.columns = pd.MultiIndex.from_product([['logreturn'], logret.columns])
# Joining logret and rawdata 
rawdata = rawdata.join(logret)
# row with the index of 2013-12-31 will be deleted as it is out of the scope of our data (2014~2017)
rawdata = rawdata.drop(pd.Timestamp('2013-12-31'))

In [14]:
print("After preprocessing the data, we have idenified the log returns of {} companies, which were included in S&P500 in 2017. To recap, we are examining the stock returns from 2014 to 2017. Therefore, we will be examining the stock returns of {} days".format(len(logret.columns), len(logret)))

After preprocessing the data, we have idenified the log returns of 438 companies, which were included in S&P500 in 2017. To recap, we are examining the stock returns from 2014 to 2017. Therefore, we will be examining the stock returns of 1010 days


#### Creating a new columns for the industry information

In [15]:
# Obtaining the information about the tickers that are included in SP_ticker_2017
industry_dic = dict()
for item in SP_ticker_2017:
    try:
        industry_dic[item] = yf.Ticker(item).info['industry']
    except:
        industry_dic[item] = None

In [19]:
# Adding the industry to a level of the column
rawdata.columns = pd.MultiIndex.from_tuples([(value, industry_dic[ticker], ticker) for value, ticker in rawdata.columns])
rawdata.head()

ValueError: too many values to unpack (expected 2)

#### Creating a new columns for days of the week

In [17]:
# The day of the week is added as a new index (creating a hierarchical index)
rawdata['days'] = [calendar.day_name[day.weekday()] for day in rawdata.index]

#### Exporting dataframe as csv

In [18]:
rawdata.to_csv('SP500_2014_2017_multilevel.csv')

#### Staking the Closing and Log returns into one dataframe

For convenience in running some of the statistical analyses, the multilevel data of stock values are stacked into a dataframe. 

In [49]:
# To easily stack the data, it is easier to delete the days and industry information.
rawdata = rawdata.drop('days', axis =1)
rawdata.columns = rawdata.columns.droplevel(1)

In [50]:
# The data is stacked 
stacked_rawdata = rawdata.stack()
stacked_rawdata.reset_index(inplace=True)
stacked_rawdata = stacked_rawdata.rename(columns = {'level_1':'Ticker'})

# Days are added as a new column
stacked_rawdata['days'] = [calendar.day_name[day.weekday()] for day in stacked_rawdata['Date']]

# Industry information is added as a new column
stacked_rawdata['Industry'] = [industry_dic[ticker] for ticker in stacked_rawdata['Ticker']]
stacked_rawdata.head()

,Date,Ticker,Adj Close,Close,High,Low,Open,Volume,logreturn,days,Industry
0,2014-01-02,A,37.592243,40.207439,40.844063,40.164520,40.844063,2678848.0,-0.017284,Thursday,Diagnostics & Research
1,2014-01-02,AAL,23.907927,25.360001,25.820000,25.059999,25.070000,8997900.0,0.004347,Thursday,Airlines
2,2014-01-02,AAP,107.652184,109.739998,111.879997,109.290001,110.360001,542700.0,-0.008529,Thursday,Specialty Retail
3,2014-01-02,AAPL,17.568451,19.754642,19.893929,19.715000,19.845715,234684800.0,-0.014164,Thursday,Consumer Electronics
4,2014-01-02,ABBV,38.174229,51.980000,52.330002,51.520000,52.119999,4569100.0,-0.015842,Thursday,Drug Manufacturers—General


#### Exporting dataframe as csv

In [51]:
stacked_rawdata.to_csv('SP500_2014_2017_stacked.csv')

-----

## Tackling Objective 2

The second objective is to examine the Monday effect on the lastest S&P 500 constituents. The process of obtaining the preprocessing the data is identical to that used for S&P500 for 2017.

### Creating a dataframe of stock returns for the latest S&P 500 constituents
The list of constituents for the latest S&P500 are available in `list of sp500.xlsx` file. 

In [5]:
SP500list = pd.read_excel('list of sp500.xlsx', engine='openpyxl')
SP_ticker_2020 = SP500list['Symbol'].tolist()

### Obtaining the stock data of latest S&P 500 constituents

In [7]:
# Using the ticker to obtain stock prices from yfinance
rawdata = yf.download(SP_ticker_2020, start="2017-12-30", end="2021-05-28")
rawdata.columns = rawdata.columns.set_names(['Value', 'Symbol'])
rawdata.head()

[*********************100%***********************]  505 of 505 completed

2 Failed downloads:
- BRK.B: No data found, symbol may be delisted
- BF.B: No data found for this date range, symbol may be delisted


Value       Adj Close                                                          \
Symbol              A        AAL         AAP       AAPL       ABBV        ABC   
Date                                                                            
2017-12-29        NaN        NaN         NaN        NaN        NaN        NaN   
2018-01-01        NaN        NaN         NaN        NaN        NaN        NaN   
2018-01-02  65.777748  51.647556  104.610184  41.310070  81.751373  88.406036   
2018-01-03  67.451401  51.014027  105.556801  41.302879  83.030678  88.735046   
2018-01-04  66.945419  51.335667  109.451698  41.494736  82.557167  88.537621   

Value                                                      ...     Volume  \
Symbol            ABMD        ABT         ACN        ADBE  ...        XEL   
Date                                                       ...              
2017-12-29         NaN        NaN         NaN         NaN  ...        NaN   
2018-01-01         NaN        NaN         NaN         NaN  ...        NaN   
2018-01-02  192.490005  55.480598  145.921112  177.699997  ...  2443400.0   
2018-01-03  195.820007  55.603271  146.594574  181.039993  ...  4114900.0   
2018-01-04  199.250000  55.508900  148.330338  183.220001  ...  2807000.0   

Value                                                                         \
Symbol           XLNX         XOM       XRAY       XYL        YUM        ZBH   
Date                                                                           
2017-12-29        NaN         NaN        NaN       NaN        NaN        NaN   
2018-01-01        NaN         NaN        NaN       NaN        NaN        NaN   
2018-01-02  2579900.0  11469300.0  1622300.0  877800.0  1747800.0  1765300.0   
2018-01-03  2453300.0  13957700.0  1533300.0  778800.0  2554900.0  1328800.0   
2018-01-04  3346500.0  10863000.0  1052400.0  796500.0  1971200.0  1073200.0   

Value                                       
Symbol          ZBRA       ZION        ZTS  
Date                                        
2017-12-29       NaN        NaN        NaN  
2018-01-01       NaN        NaN        NaN  
2018-01-02  310600.0  2387100.0  2135600.0  
2018-01-03  253000.0  1575700.0  2328200.0  
2018-01-04  435200.0  3309200.0  2534000.0  

[5 rows x 3030 columns]

#### Deleting the missing values

In [8]:
# Inspecting the missing values in terms of rows
rawdata['Close'].isna().sum(axis=0).describe()

count    505.000000
mean      12.192079
std       73.824631
min        3.000000
25%        3.000000
50%        3.000000
75%        3.000000
max      860.000000
dtype: float64

In [9]:
# Making an list of tuples for tickers that has more than 3 missing values
high_missing_ticker = rawdata['Close'].isna().sum(axis=0) > 3
high_missing_ticker_list = high_missing_ticker[high_missing_ticker].index.tolist()
high_missing_ticker_tuples = list()
for i in ['Adj Close', 'Open', 'Close', 'High' ,'Low', 'Volume']:
    high_missing_ticker_tuples += list(zip([i]*len(high_missing_ticker_list),high_missing_ticker_list))

In [10]:
# Excluding columns (i.e., tickers) that has more than 3 missing values 
rawdata = rawdata.drop(high_missing_ticker_tuples, axis = 1)

In [13]:
# Finding out the dates that all tickers (columns) have missing values
missingdate =rawdata.isna().sum(axis=1) > 0
missingdate[missingdate].index

DatetimeIndex(['2017-12-29', '2018-01-01', '2018-12-05'], dtype='datetime64[ns]', name='Date', freq=None)

In [21]:
# row with the index of 2017-12-29 will be deleted as it is out of the scope of our data
rawdata = rawdata.drop(pd.Timestamp('2017-12-29'))

KeyError: "[Timestamp('2017-12-29 00:00:00')] not found in axis"

In [24]:
# rwo with the index of 2018-01-01 is deleted as it is missing values for all tickers
rawdata = rawdata.drop(pd.Timestamp('2018-01-01'))

KeyError: "[Timestamp('2018-01-01 00:00:00')] not found in axis"

In [23]:
# row with the index of 2018-12-05 is replaced with the values from the previous date
rawdata = rawdata.fillna(method= 'ffill')

In [25]:
# Double check to see if all the missing values were either removed or replaced
(rawdata.isna().sum(axis=None)>0).any()

False

#### Calculating the log returns for closing price

In [28]:
# Getting the log returns from stock prices
logret = np.log(rawdata['Close']).diff()
logret.columns = pd.MultiIndex.from_product([['logreturn'], logret.columns])
# Joining logret and rawdata 
rawdata = rawdata.join(logret)

ValueError: columns overlap but no suffix specified: MultiIndex([('logreturn',    'A'),
            ('logreturn',  'AAL'),
            ('logreturn',  'AAP'),
            ('logreturn', 'AAPL'),
            ('logreturn', 'ABBV'),
            ('logreturn',  'ABC'),
            ('logreturn', 'ABMD'),
            ('logreturn',  'ABT'),
            ('logreturn',  'ACN'),
            ('logreturn', 'ADBE'),
            ...
            ('logreturn',  'XEL'),
            ('logreturn', 'XLNX'),
            ('logreturn',  'XOM'),
            ('logreturn', 'XRAY'),
            ('logreturn',  'XYL'),
            ('logreturn',  'YUM'),
            ('logreturn',  'ZBH'),
            ('logreturn', 'ZBRA'),
            ('logreturn', 'ZION'),
            ('logreturn',  'ZTS')],
           length=495)

In [30]:
print("After preprocessing the data, we have idenified the log returns of {} companies, which were included in S&P500 in 2021. To recap, we are examining the stock returns from 2018 to May 2021. Therefore, we will be examining the stock returns of {} days".format(len(logret.columns), len(logret)))

After preprocessing the data, we have idenified the log returns of 495 companies, which were included in S&P500 in 2021. To recap, we are examining the stock returns from 2018 to May 2021. Therefore, we will be examining the stock returns of 857 days


#### Creating a new columns for the industry information

In [32]:
# Obtaining the information about the tickers that are included in SP_ticker_2017
industry_dic = dict()
for item in SP_ticker_2020:
    try:
        industry_dic[item] = yf.Ticker(item).info['industry']
    except:
        industry_dic[item] = None

In [33]:
# Adding the industry to a level of the column
rawdata.columns = pd.MultiIndex.from_tuples([(value, industry_dic[ticker], ticker) for value, ticker in rawdata.columns])
rawdata.head()

Adj Close                              \
           Diagnostics & Research   Airlines Specialty Retail   
                                A        AAL              AAP   
Date                                                            
2018-01-02              65.777748  51.647556       104.610184   
2018-01-03              67.451401  51.014027       105.556801   
2018-01-04              66.945419  51.335667       109.451698   
2018-01-05              68.015762  51.316177       110.615242   
2018-01-08              68.161713  50.809345       109.836258   

                                                            \
           Consumer Electronics Drug Manufacturers—General   
                           AAPL                       ABBV   
Date                                                         
2018-01-02            41.310070                  81.751373   
2018-01-03            41.302879                  83.030678   
2018-01-04            41.494736                  82.557167   
2018-01-05            41.967163                  83.994331   
2018-01-08            41.811283                  82.648544   

                                                            \
           Medical Distribution Medical Devices              
                            ABC            ABMD        ABT   
Date                                                         
2018-01-02            88.406036      192.490005  55.480598   
2018-01-03            88.735046      195.820007  55.603271   
2018-01-04            88.537621      199.250000  55.508900   
2018-01-05            89.609322      202.320007  55.669323   
2018-01-08            91.094681      207.800003  55.508900   

                                                                    ...  \
           Information Technology Services Software—Infrastructure  ...   
                                       ACN                    ADBE  ...   
Date                                                                ...   
2018-01-02                      145.921112              177.699997  ...   
2018-01-03                      146.594574              181.039993  ...   
2018-01-04                      148.330338              183.220001  ...   
2018-01-05                      149.553955              185.339996  ...   
2018-01-08                      150.749115              185.039993  ...   

                              logreturn                                      \
           Utilities—Regulated Electric Semiconductors Oil & Gas Integrated   
                                    XEL           XLNX                  XOM   
Date                                                                          
2018-01-02                          NaN            NaN                  NaN   
2018-01-03                    -0.006716       0.019837             0.019450   
2018-01-04                    -0.007822       0.017892             0.001383   
2018-01-05                    -0.007028       0.050619            -0.000807   
2018-01-08                     0.007452       0.006586             0.004486   

                                                                          \
           Medical Instruments & Supplies Specialty Industrial Machinery   
                                     XRAY                            XYL   
Date                                                                       
2018-01-02                            NaN                            NaN   
2018-01-03                      -0.003431                       0.012120   
2018-01-04                      -0.000149                       0.006654   
2018-01-05                       0.013953                      -0.001876   
2018-01-08                       0.006758                       0.003605   

                                                                               \
           Restaurants Medical Devices Communication Equipment Banks—Regional   
                   YUM             ZBH                    ZBRA           ZION   
Date

#### Creating a new columns for days of the week

In [34]:
# The day of the week is added as a new index (creating a hierarchical index)
rawdata['days'] = [calendar.day_name[day.weekday()] for day in rawdata.index]

#### Exporting dataframe as csv

In [35]:
rawdata.to_csv('SP500_2018_2021_multilevel.csv') 

#### Staking the Closing and Log returns into one dataframe

For convenience in running some of the statistical analyses, the multilevel data of stock values are stacked into a dataframe. 

In [36]:
# To easily stack the data, it is easier to delete the days and industry information.
rawdata = rawdata.drop('days', axis =1)
rawdata.columns = rawdata.columns.droplevel(1)

In [51]:
# The data is stacked 
stacked_rawdata = rawdata.stack()
stacked_rawdata.reset_index(inplace=True)
stacked_rawdata = stacked_rawdata.rename(columns = {'level_1':'Ticker'})

# Days are added as a new column
stacked_rawdata['days'] = [calendar.day_name[day.weekday()] for day in stacked_rawdata['Date']]

# Industry information is added as a new column
stacked_rawdata['Industry'] = [industry_dic[ticker] for ticker in stacked_rawdata['Ticker']]
stacked_rawdata.head()

# Before we forget, let's add the days and industry column back to our rawdata. We will comeback to this later
rawdata.columns = pd.MultiIndex.from_tuples([(value, industry_dic[ticker], ticker) for value, ticker in rawdata.columns])
rawdata['days'] = [calendar.day_name[day.weekday()] for day in rawdata.index]


KeyError: ''

#### Exporting dataframe as csv

In [38]:
stacked_rawdata.to_csv('SP500_2018_2021_stacked.csv')

### Analyzing the Monday Effect
The next step is to analyze the Monday effect. First, as done in Arman and Lestari's research, one-sample t-test is conducted for each day of the week. The test value is 0. Therefore, a significant result indicates that it is highly unlikely to have obtained the average log stock returns on a specific day of the week given that the null hypothesis is true (i.e. the average log stock return is 0).

##### Descriptive Statistics

In [97]:
# Let's examine how many columns we have
print('There are {} columns in our dataset'.format(len(rawdata.columns)))
# The days of the week is one of the columns
print('Is days one of the columns?','days' in rawdata.columns)

There are 3465 columns in our dataset
Is days one of the columns? False


In [93]:
# Before we move on, let's make a multilevel index for the rows.
rawdata.set_index('days', append=True, inplace=True)

KeyError: "None of ['days'] are in the columns"

In [117]:
# Frequency count for each day
rawdata['logreturn'].groupby(level=1).count().mean(axis=1)

days
Friday       170.0
Monday       163.0
Thursday     174.0
Tuesday      175.0
Wednesday    174.0
dtype: float64

In [94]:
# Before we get into conducting one sample t-test, 
# let's look at the mean of log stock returns for each day of the week
rawdata['logreturn'].groupby(level=1).mean().mean(axis=1).sort_values()

days
Thursday    -0.000048
Monday      -0.000022
Wednesday    0.000064
Friday       0.001077
Tuesday      0.001108
dtype: float64

In [95]:
# Let's look at the standard deviation of log stock returns for each day of the week
rawdata['logreturn'].groupby(level=1).std().std(axis=1).sort_values()

days
Tuesday      0.005929
Friday       0.006209
Thursday     0.006756
Wednesday    0.007444
Monday       0.008791
dtype: float64

Although less tha Thursday, Monday has a negative log stock returns and a largest variance. Let's see if this value is statistically significant. 

##### Inferential Statistics

###### t-test

In [149]:
def one_sample_t(day):
    a = rawdata.xs(day, level='days').logreturn.values.flatten()
    a = a[~numpy.isnan(a)]
    return stats.ttest_1samp(a,0)

In [150]:
# one-sample t-test for Monday
one_sample_t('Monday')

Ttest_1sampResult(statistic=-0.21902829372216906, pvalue=0.8266285616536038)

In [151]:
# one-sample t-test for Tuesday
one_sample_t('Tuesday')

Ttest_1sampResult(statistic=14.19998541482364, pvalue=1.0314077682210805e-45)

In [152]:
# one-sample t-test for Wednesday
one_sample_t('Wednesday')

Ttest_1sampResult(statistic=0.7698756095608359, pvalue=0.4413757947758502)

In [153]:
# one-sample t-test for Thursday
one_sample_t('Thursday')

Ttest_1sampResult(statistic=-0.5764213993864189, pvalue=0.5643318898456581)

In [154]:
# one-sample t-test for Friday
one_sample_t('Friday')

Ttest_1sampResult(statistic=14.226349283101952, pvalue=7.10881134086702e-46)

----------

## Tackling Objective 2 <a clas ="anchor" id = "tackling-objective-2"></a>

In [2]:
import rpy2
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
